# Semantic search index creation 

### *Guide*

Anton Antonov    
September 2024  

-----

## Introduction

This notebook shows how to create an LLM-computed vector database over the paragraphs of relative large text.

-------

## Setup

In [1]:
use Data::Importers;
use LLM::Configurations;
use LLM::Functions;
use XDG::BaseDirectory :terms;

use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Reshapers;
use Data::Summarizers;
use Math::Nearest;
use Math::DistanceFunctions;
use Statistics::OutlierIdentifiers;

------

## Ingest text

Ingest the transcript of the (3.5 hours) discussion [CWv1]:

In [2]:
my $url = 'https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution';
my $txtEN = data-import($url, 'plaintext');

text-stats($txtEN)

(chars => 245233 words => 36863 lines => 7107)

Take the "proper transcript" part:

In [3]:
my $txtEN2 = $txtEN.substr($txtEN.index('Starting point is 00:00:00'));
text-stats($txtEN2)

(chars => 242067 words => 36490 lines => 7048)

Split into paragraphs and make the paragraphs compact:

In [4]:
my @paragraphs = $txtEN2.split(/ 'Starting point is' \h+ [\d ** 2]+ % ':' /):g;
@paragraphs .= map({ $_.subst(/\n+/, "\n"):g});
@paragraphs.elems

442

Show a sample of the paragraphs:

In [5]:
#% html
@paragraphs.pick(4) ==> to-html()

of them, they're brilliant. Their hearts are in the right place. But if we get into an
argument about Ovo testes, they're going to lose as two biologists and I'm going to win.
It's not all about motility of gametes. Uh, the world will keep throwing curve balls at
you and you have to begin from a heart open place to say, some of us are shit out of
luck because we fall in edge categories.
And so I stand by everything that you and I did last time.
It's a difficult place to be.
And if you have to simplify it as to boys or boys, women or women, you're not getting
 
 card, but somehow magician makes sure that the card
that you pick is the one that he knows.
Uh, in that situation, you have magician's choice
in the primaries, and then you'd have the duopoly
field, two candidates, either of which was
acceptable and you could actually afford to hold
an election and the populace would vote.
And that way the international order wasn't put at risk. And you could actually afford to hold an election
 
 Uh, you're running the packet.
Done.
There's the lefty innervation packet.
And it's something that you're sort of obligated to run,
if you're going to be a member in good standing
of the American left.
So one part of it would say that correlation
does not imply causation.
 
 There was a union organizer whose house, I think,
was shot up by a detective agency
to intimidate him into not organizing the workers.
And his wife stayed in the house and penned the song.
Which side are you on boys?
Which side are you on?
They say in Highland County, there are no neutrals there.
You'll either be a union man or thug for J.H. Blair.

------

## Make vector database

Make an empty vector database object:

In [6]:
my $vdbObj = LLM::RetrievalAugmentedGeneration::VectorDatabase.new(name => 'No833');

VectorDatabase(:id("603118ff-5738-4328-92cc-7aa2a261714d"), :name("No833"), :elements(0), :sources(0))

Make an LLM access specification:

In [12]:
#my $conf = llm-configuration("ChatGPT", model => 'text-embedding-002');
my $conf = llm-configuration("Gemini");

$conf.Hash.elems

24

Create the semantic index for the vector database object (an profile it):

In [13]:
my $tstart = now;
$vdbObj.create-semantic-search-index(@paragraphs, method => 'by-max-tokens', max-tokens => 2048, e => $conf):embed;
my $tend = now;
say "Time to make the semantic search index: {$tend - $tstart} seconds.";

Time to make the semantic search index: 211.465842025 seconds.


By default the vector database object is exported in a sub-directory of [`$XDG_DATA_HOME`](https://specifications.freedesktop.org/basedir-spec/latest/index.html):

In [31]:
# The sub-directory
my $dirname = data-home.Str ~ '/raku/LLM/SemanticSearchIndex';

# The exported vector database base file name
my $basename = "SemSe-{$vdbObj.id}.json";

# Corresponding IO:Path object
my $file = IO::Path.new(:$dirname, :$basename);

# Check for existence
$file.f

True

The export path is saved in the vector database object:

In [ ]:
$file.Str eq $vdbObj.location

Show a sample of the text chunks:

In [39]:
#% html
$vdbObj.text-chunks.pairs.pick(4).sort(*.key) ==> to-html()

092.0,"What if the idea is that an outbreak of truth and democracy would destroy NATO and the world order? Let's imagine that that would undo the markets that would spread nukes. You know, what happens if, if ending, uh, the control of social media would mean that weaponized anthrax plant plans could be spread frictionlessly. If four amino acids lead to worldwide lockdowns,"
177.0,"a lot of what you're doing is just very, very low quality. But this thing that you're doing over here, I'll vouch for it, I'll put my name behind that. That's really clever and really good. If I can't find one jewel, one gem, one positive thing to say, it's, it's for somebody else. And part of this is I'm trying to indicate, this is what criticism actually is. I'm modeling what criticism. I wasn't weak. I didn't shy away from what I considered to be the sort of uninformed or pseudoscientific or historically"
266.0,"But then if you were to see Douglas Murray and Malcolm Gladwell on stage together, or Ben Shapiro and anybody, and you go, they're able to be disagreeable so seamlessly. For me to get even 5% of the way there, I need to do the equivalent of a one rep max to ask Abigail Shrier about CBT. And that's for me just an obvious area"
323.0,"Every second of my life spent in your classrooms before college, before university is a second I want back is trauma is pain. All you did is instill in me that I'm an idiot. I'm a moron. I'm not good enough. I should go away. I'm bad. I'm aberrant. It's like, I got it. I really got idiot. I'm a moron. I'm not good enough. I should go away. I'm bad."


Show dimensions and data type of the obtained vectors:

In [ ]:
say "dimensions : ", $vdbObj.database.&dimensions;
say "data type  : ", deduce-type($vdbObj.database)

-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[CWv1] Chris Williamson,
["Eric Weinstein - Are We On The Brink Of A Revolution? (4K)"](https://www.youtube.com/watch?v=PYRYXhU4kxM),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution).)